# Compare with random

In [3]:
import sys
try:
    sys.path.insert(0, "/usr/lib/python3.7/site-packages")
except FileNotFoundError:
    pass
import torch as t
import pandas as pd
import math
import random
from allennlp.predictors.predictor import Predictor

In [53]:
def softmax(tens: t.Tensor):
    tmp = []
    denom = float(t.sum(t.exp(tens)))
    for i in range(tens.size()[0]):
        tmp += [math.exp(tens[i])/denom]
    return t.Tensor(tmp)

def cross_entropy_softmax(gold_prob: t.Tensor, mod_prob: t.Tensor):
    gold_prob = softmax(gold_prob.float())
    mod_prob  = softmax(mod_prob.float())
    # sum(p*log(q))
    return float(-t.sum(t.mul( gold_prob, t.log2(mod_prob) )))

def KL_divergence_softmax(gold_prob: t.Tensor, mod_prob: t.Tensor):
    gold_prob = softmax(gold_prob.float())
    mod_prob  = softmax(mod_prob.float())
    # sum(p*log(q/p))
    return float(-t.sum(t.mul( gold_prob, t.log2(t.div(mod_prob, gold_prob)) )))

def cross_entropy(gold_prob: t.Tensor, mod_prob: t.Tensor):
    denom1 = float(t.sum(gold_prob))
    denom2 = float(t.sum(mod_prob))
    gold_prob = t.div(gold_prob.float(), denom1)
    mod_prob  = t.div(mod_prob.float(), denom2)
    return float(-t.sum(t.mul(gold_prob, t.log2(mod_prob))))

def KL_divergence(gold_prob: t.Tensor, mod_prob: t.Tensor):
    denom1 = float(t.sum(gold_prob))
    denom2 = float(t.sum(mod_prob))
    gold_prob = t.div(gold_prob.float(), denom1)
    mod_prob  = t.div(mod_prob.float(), denom2)
    return float(-t.sum(t.mul(gold_prob, t.log2(t.div(mod_prob, gold_prob)))))
        
def str2list(x):
    return [y.strip() for y in x[2:-2].replace('"','').replace("'",'').replace(".", '').split(',')]
def list2set(x):
    return set(x)

def uniq_events (test_df, dictionary):
    test_df["Xintent"] = test_df["Xintent"].apply(str2list)
    test_df["Xemotion"] = test_df["Xemotion"].apply(str2list)
    test_df["Otheremotion"] = test_df["Otheremotion"].apply(str2list)
    if(dictionary is not None):
        dictionary = []
        dictionary += (test_df["Xintent"].sum() + test_df["Xemotion"].sum() + test_df["Otheremotion"].sum())
        dictionary = set(dictionary)
        dictionary = list(dictionary)
    test_df = test_df.groupby(["Event"])["Xintent", "Xemotion", "Otheremotion"].sum()
    
    test_df["Xintent"] = test_df["Xintent"].apply(list2set)
    test_df["Xemotion"] = test_df["Xemotion"].apply(list2set)
    test_df["Otheremotion"] = test_df["Otheremotion"].apply(list2set)
    if(dictionary is not None):
        return test_df, dictionary
    else:
        return test_df

In [54]:
def rand_output(dictionary):
    n = random.randrange(1,10)
    out = []
    for i in range(n):
        out += [dictionary[random.randrange(0,len(dictionary))]]
    out += ["none"]
    return out

## Metrics using softmax

In [55]:
def metrics_with_softmax(predictor, rand_gen, path_to_csv = 'event2mind/test.csv'):
    gold_ds = pd.read_csv(path_to_csv)[['Event','Xintent','Xemotion','Otheremotion']]
    dictionary = []
    if(not rand_gen):
        dictionary = None
        gold_ds = uniq_events(gold_ds, dictionary)
    else:
        gold_ds, dictionary = uniq_events(gold_ds, dictionary)
    connected = {"Xintent": "xintent", "Xemotion": "xreact", "Otheremotion": "oreact"}
    average = 0
    L = gold_ds.shape[0]
    counter = 0; n = 0
    
    gold_res = []
    mod_res = []
    for index, row in gold_ds.iterrows():
        if(n == (counter*100)//L):
            print("\r" + str(n) + "%" + " " + "#"*(n//2) + "_"*(50 - n//2), end = "")
            n += 1
        for column_name in connected.keys():
            result = predictor.predict(source=index)
            model_out = [ (" ".join(react), prob) for react, prob in zip(result[connected[column_name] + "_top_k_predicted_tokens"], result[connected[column_name] + "_top_k_log_probabilities"])]
            if(rand_gen):
                tmp = rand_output(dictionary)
                gold_res += [len(row[column_name])]
                mod_res  += [len(set(tmp) & row[column_name])]
            else:
                gold_res += [len(row[column_name])]
                mod_res  += [len(set([word[0] for word in model_out[:10]]) & row[column_name])]
        counter  += 1
    print("\r", end = "")
    return [KL_divergence_softmax(t.Tensor(gold_res), t.Tensor(mod_res)), cross_entropy_softmax(t.Tensor(gold_res), t.Tensor(mod_res))]

## Metrics without softmax

In [56]:
def metrics(predictor, rand_gen, path_to_csv = 'event2mind/test.csv'):
    gold_ds = pd.read_csv(path_to_csv)[['Event','Xintent','Xemotion','Otheremotion']]
    dictionary = []
    if(not rand_gen):
        dictionary = None
        gold_ds = uniq_events(gold_ds, dictionary)
    else:
        gold_ds, dictionary = uniq_events(gold_ds, dictionary)
    connected = {"Xintent": "xintent", "Xemotion": "xreact", "Otheremotion": "oreact"}
    average = 0
    L = gold_ds.shape[0]
    counter = 0; n = 0
    
    gold_res = []
    mod_res = []
    for index, row in gold_ds.iterrows():
        if(n == (counter*100)//L):
            print("\r" + str(n) + "%" + " " + "#"*(n//2) + "_"*(50 - n//2), end = "")
            n += 1
        for column_name in connected.keys():
            result = predictor.predict(source=index)
            model_out = [ (" ".join(react), prob) for react, prob in zip(result[connected[column_name] + "_top_k_predicted_tokens"], result[connected[column_name] + "_top_k_log_probabilities"])]
            if(rand_gen):
                tmp = rand_output(dictionary)
                gold_res += [len(row[column_name])]
                mod_res  += [len(set(tmp) & (row[column_name] | set(["none"])))]
            else:
                gold_res += [len(row[column_name])]
                mod_res  += [len(set([word[0] for word in model_out[:10]] + ["none"]) & (row[column_name] | set(["none"])))]
        counter  += 1
    print("\r", end = "")
    return [KL_divergence(t.Tensor(gold_res), t.Tensor(mod_res)), cross_entropy(t.Tensor(gold_res), t.Tensor(mod_res))]

In [57]:
predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/event2mind-2018.10.26.tar.gz")

KL_div, cross_entr = metrics(predictor, rand_gen=False)
print("Model  Cross-Entropy: ", str(float('{:.4f}'.format(cross_entr))), ' '*25)
print("Model Kullback-Leibler divergence:", str(float('{:.4f}'.format(KL_div))), ' '*25, "\n")

KL_div, cross_entr = metrics(predictor, rand_gen=True)
print("Random Cross-Entropy: ", str(float('{:.4f}'.format(cross_entr))), ' '*25)
print("Random Kullback-Leibler divergence:", str(float('{:.4f}'.format(KL_div))), ' '*25, "\n")

Model  Cross-Entropy:  12.7337                          
Model Kullback-Leibler divergence: 0.2267                           

Random Cross-Entropy:  12.7266                          
Random Kullback-Leibler divergence: 0.2196                           



In [46]:
predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/event2mind-2018.10.26.tar.gz")
KL_div, cross_entr = metrics_with_softmax(predictor, rand_gen=False)
print("Softmaxed:\n")
print("Model  Cross-Entropy:              ", str(float('{:.4f}'.format(cross_entr))), ' '*25)
print("Model  Kullback-Leibler divergence:", str(float('{:.4f}'.format(KL_div))), ' '*25, "\n")

KL_div, cross_entr = metrics_with_softmax(predictor, rand_gen=True)
print("Random Cross-Entropy:              ", str(float('{:.4f}'.format(cross_entr))), ' '*25)
print("Random Kullback-Leibler divergence:", str(float('{:.4f}'.format(KL_div))), ' '*25)

Model  Cross-Entropy:               12.8566                     
Model  Kullback-Leibler divergence: 1.9641                     
Random Cross-Entropy:               13.224                     
Random Kullback-Leibler divergence: 2.3315                     
